# GiveDirectly Take Home Assessment

## Question 1

1. Please evaluate the data in recipients.csv and survey_attempts.csv to answer the following questions:
    1. How many recipients are in each of the four stages? Please provide the calculation(s) in the spreadsheet or code that you submit.
    2. How many surveys were successfully completed in December, 2020? Please provide the calculation(s) in the spreadsheet or code that you submit.
    3. Did you find any abnormalities in the source data? If so, how did you account for them in your analysis?

First install the required packages.

In [ ]:
!pip install -r requirements.txt

Import the required libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from sklearn.preprocessing import LabelEncoder
matplotlib.use('QtAgg')

from statsmodels.stats.weightstats import CompareMeans
import statsmodels.formula.api as smf
import missingno as msno
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.model_selection import train_test_split
from collections import defaultdict

Let's now import the two datasets and merge them

In [ ]:
recipients = pd.read_csv('recipients.csv')
attempts = pd.read_csv('survey_attempts.csv')
merged = pd.merge(recipients, attempts, on='recipient_id', how='left', indicator=True)
merged._merge.value_counts(dropna=False)


It looks like all the recipients are matched with survey attempts. We can drop the `_merge` variable

In [ ]:
merged.drop(columns='_merge', inplace=True)

A quick glance at our data

In [ ]:
merged.head(10)

Check for duplicates. We expect that `recipient_id` and `survey_id` together form a unique id.


In [ ]:
merged[merged.duplicated(subset=['recipient_id', 'survey_id'], keep=False)]

There are 50 pairs of duplicates that need to be dropped.  

In [ ]:
merged = merged.drop_duplicates(subset=['recipient_id', 'survey_id'], keep='first').reset_index(drop=True)


Let's now calculate the stage variable, starting with 'Start'.

In [ ]:
mask = merged.groupby('recipient_id')['success'].any()
one_success = [mask.index[i] for i, m in enumerate(mask) if m]
# Set the value to start for those with no successful surveys. '~' negates the value 
# of the mask. In this case, ~mask means find those without any successful survey
merged.loc[~merged.recipient_id.isin(one_success), 'stage'] = 'Start'

Next, the 'Ineligible' stage

In [ ]:
# Remove the text 'County' from the column
merged.county = merged.county.str.replace('County ', '', regex=False)
    
inABC = merged.county.isin(['A', 'B', 'C'])
recipient_noABC = merged.recipient_id[~inABC]

merged.loc[merged.recipient_id.isin(one_success) &
    merged.recipient_id.isin(recipient_noABC), 'stage'] = 'Ineligible'

The 'Review' stage

In [ ]:
recipient_yesABC = merged.recipient_id[inABC]
notActive = merged.account_status == 'Not Active'
recipient_notActive = merged.recipient_id[notActive]

merged.loc[merged.recipient_id.isin(one_success) &
           merged.recipient_id.isin(recipient_yesABC) &
           merged.recipient_id.isin(recipient_notActive),
           'stage'] = 'Review'

And finally the 'Pay' stage

In [ ]:
active = merged.account_status == 'Active'
recipient_active = merged.recipient_id[active]
merged.loc[merged.recipient_id.isin(one_success) &
           merged.recipient_id.isin(recipient_yesABC) &
           merged.recipient_id.isin(recipient_active), 'stage'] = 'Pay'

### 1.1 How many recipients in each stage?

In [ ]:
merged.stage.value_counts(dropna=False)

There are 209 in the 'Pay' stage, 150 in the 'Start', 150 'Ineligible', 35 in 'Review', and 6 not in any stage.
This is because there are 6 recipients with a missing `account_status`. 

In [ ]:
merged.loc[merged.stage.isna()]

We will impute a value for recipients with a missing `account_status`,
but it could be worthwhile to follow up with someone for the appropriate status.

### 1.2 How many successful surveys in December?

In [ ]:
merged['month'] = merged['date'].astype(str).str[:2]
merged.groupby('month')['success'].sum()

There were 93 successful surveys in December.

### 1.3 Abnormalities in the source data

There were duplicates in the data that had to be dropped. Some values also did not make much sense and were flagged as missing. The missing data was later imputed.

In [ ]:
merged.describe()

`time_county` has a minimum of -45 and `age` has a max of 9999. To deal with these problematic cases, I replace them with missing values to be imputed later. It could also be good to follow up with other teams on the correct value.

In [ ]:

merged.age[merged.age == 9999] = pd.np.nan 
merged.time_county[merged.time_county < 0] = pd.np.nan

There are missing values in the data as well.

In [ ]:
merged.isna().sum()

Six columns have missing data.

Missing data is usually deleted or imputed. Deleting missing data is easiest, but it can lead to biases if the data is not missing at random. If `age` is more likely to be missing in certain counties, for example, deleting missing data isn't the best approach. If the data is missing completely at random or missing at random, it can be deleted.

If there are many missing values in a row, the row can be deleted. Likewise, if there are many missing values in a column, the column can be deleted. 

Imputation can be as simple as replacing the missing value in a column with an arbitrary value such as '0' or the column mean or mode. Other techniques take into account the values in other columns. Suppose a recipient has a missing `age`, but lives in a certain county and has also responded to the survey. Then the `age` might be imputed to be slightly lower because of a correlation between `success` and `age` or `county` and `age`. See more [here](https://www.analyticsvidhya.com/blog/2021/10/handling-missing-value/#h2_2).

We can investigate the missing data with the `missingno` package.

In [ ]:
%matplotlib inline

msno.matrix(merged)
plt.xticks(rotation=15)
plt.tight_layout()
plt.show()

The white lines represent missing values. The `account_number` and `account_status` appear to have coinciding missing values, and the same for `age` and `time_county`. We can visualize these relationships with a heatmap.

In [ ]:
%matplotlib inline

msno.heatmap(merged, cmap='rainbow')
plt.tight_layout()
plt.show()

There is a strong correlation between `account_number` and `account_status` being missing, which
would make sense considering that someone without an `account_number` would not have 
an `account_status` and vice versa. There is also a strong association between `age` and `time_county` missingness.

To investigate this, let's look at recipients with a missing `time_county`.

In [ ]:
time_county_missing = merged.loc[merged.time_county.isna()]
print(time_county_missing.head())

In [ ]:
age_missing = merged.loc[merged.age.isna()]
print(age_missing.head())

In [ ]:
print(time_county_missing.recipient_id.nunique())

In [ ]:
print(age_missing.recipient_id.nunique())

So `age` is usually missing when `time_county` is missing. There are five recipients that did not fill in their `age` and four that did not fill in `time_county` for multiple survey attempts. They may have been uncomfortable sharing that information.

Dropping these records could lead to bias because they are not missing at random. Let's impute the `age` and `time_county` for them. We'll also impute the `county` and `account_status`, which are categorical variables. The imputation strategy will regress the column with missing data, say `age`, on other columns in the data. It will then use the regression coefficients to predict the missing values for `age`. More details can be found [here](https://scikit-learn.org/stable/modules/impute.html#iterative-imputer) and [here](https://www.analyticsvidhya.com/blog/2021/10/handling-missing-value/#h2_2)  

Starting with the categorical columns, we'll prepare the dataset for imputation by converting `account_status` and `county` to categorical codes.

In [ ]:
cat_cols_na = ['account_status', 'county']
merged[cat_cols_na] = merged[cat_cols_na].astype('category')
d_na = {col: {n: cat for n, cat in enumerate(merged[col].cat.categories)}
            for col in cat_cols_na}
merged[cat_cols_na] = pd.DataFrame(
    {col: merged[col].cat.codes for col in cat_cols_na},
    index=merged.index
)
print(merged[cat_cols_na].head())

Now we can impute the missing values.

In [ ]:

imp_cat = IterativeImputer(estimator=ExtraTreesClassifier(),
                            initial_strategy='most_frequent',
                            max_iter=10, random_state=0, missing_values=-1)


merged[cat_cols_na] = imp_cat.fit_transform(merged[cat_cols_na])

Finally, we convert the numerical codes back to their original labels

In [ ]:
for col in cat_cols_na:
        merged[col].replace(d_na[col], inplace=True)
print(merged[cat_cols_na].head())

`age` and `time_county` are numeric variables, so they don't need any conversion.

In [ ]:
num_cols_na = ['age', 'time_county']
imp_num = IterativeImputer(estimator=ExtraTreesRegressor(),
                               initial_strategy='median',
                               max_iter=10, random_state=0)
merged[num_cols_na] = imp_num.fit_transform(merged[num_cols_na])

Now that `account_status` has been imputed, the `stage` variable can be recalculated because the six missing cases were missing because of the missing `account_status` at the time of calculation.

In [ ]:
notActive = merged.account_status == 'Not Active'
recipient_notActive = merged.recipient_id[notActive]

merged.loc[merged.recipient_id.isin(one_success) &
           merged.recipient_id.isin(recipient_yesABC) &
           merged.recipient_id.isin(recipient_notActive),
           'stage'] = 'Review'

In [ ]:
active = merged.account_status == 'Active'
recipient_active = merged.recipient_id[active]
merged.loc[merged.recipient_id.isin(one_success) &
           merged.recipient_id.isin(recipient_yesABC) &
           merged.recipient_id.isin(recipient_active), 'stage'] = 'Pay'

Another look at which columns contain missing data shows that there is only the `account_number` column, which can be ignored for analysis. It also would not be appropriate to impute an identifier.

In [ ]:
merged.isna().sum()

## Question 2

2. The program manager has asked for data to help determine whether the field team should focus more effort on calling those in stage Start or following up to resolve issues with those in stage Review. Please write a response to the program manager, including data that may help inform the decision, and some additional factors that you would take into consideration to make the decision. Assume that the program manager’s expertise does not include interpreting data and complex analytics. Please limit your written response to 300 words or less. 

It would not make sense to calculate the chance of someone having a successful survey based on the `stage` variable because the 'Start' group would have no successful surveys and the other three groups would have all recipients with at least one successful survey. On the other hand, it would be good to know the chances of someone moving from the 'Review' stage to a 'Pay' stage at a later date, for example. 

In [ ]:
# Check how many recipients have more than one stage
(merged.groupby('recipient_id')['stage'].nunique() > 1).sum()

There do not appear to be recipients that have changed their stage. Let's check for recipients
that have changed from 'Not Active' to 'Active' `account_status`.

In [ ]:
(merged.groupby('recipient_id')['account_status'].nunique() > 1).sum()

No recipients have changed from 'Not Active' to 'Active' `account_status` either.


One thing to consider is that the 'Start' group has 150 recipients compared to only 35 in the 'Review' group.
The 'Start' group success rate only needs to be 35/150 ~ 23% to match a 100% success rate in the
'Review' group. It would be good to look up historical data on the conversion rate from 'Start' to 'Pay' versus 'Review' to 'Pay'. Data on the cost of converting the 'Start' group versus the 'Review' group would also be helpful for the decision.

In [ ]:
merged.stage.value_counts(dropna=False)

## Question 3

3. The country director is considering investing resources into proactively conducting in-person surveys with recipients *in the highest age group* across projects to increase overall survey success rate. They believe that this additional cost might outweigh the current costs of repeated failed phone survey attempts, if we can accurately target those recipients *least likely* to respond to a phone survey. 
    1. What analysis would you provide from the provided project data to help make this decision? Please provide the calculation(s) in the spreadsheet or code that you submit.
    2. Are there other factors that might explain the observed survey success rate from this project? Please use your judgment to determine these factors and limit your written response to 400 words or less. 

### 3.1

Let's graph the relationship between `age` and at least one successful survey.

In [ ]:
%matplotlib inline

merged['one_success'] = 0
merged.loc[merged.recipient_id.isin(one_success), 'one_success'] = 1

# Create bins for age broken into quartiles
merged['age_bin'] = pd.qcut(merged.age, q=4)
age_success = merged.groupby('age_bin')['one_success'].sum()
age = merged.groupby('age_bin')['one_success'].count()
results = age_success.div(age, level='age_bin') * 100

results.plot(kind='bar')

plt.xticks(rotation=0)
plt.xlabel('Age Group')
plt.ylabel('% of recipients with at least\n one successful survey')
plt.show()

People in the highest age group are the least likely to have at least one successful survey.

Using logistic regression we have

In [ ]:
logit = smf.logit('one_success ~ age', data=merged).fit()
print(logit.summary())

There is a negative relationship between `age` and the chance of at least one successful survey. The coefficient is also statistically significant. To interpret it we calculate the odds ratio.

In [ ]:
odds_ratios = pd.DataFrame({
        'OR': logit.params, 
        'Lower CI': logit.conf_int()[0],
        'Upper CI': logit.conf_int()[1]
    })
    
odds_ratios = pd.np.exp(odds_ratios)
print(odds_ratios)
print(round((odds_ratios['OR'][1] - 1) * 100, 2))

Each additional increase of one year in age is associated with a roughly 16 percent decrease
in odds of having at least one successful survey. 

So far, it appears that older recipients are less likely to respond to surveys. Focusing on the 
older recipients would help target those with lower response rates.

### 3.2

We include other variables in our logistic regression to check for confounders.

In [ ]:
big_logit = smf.logit('one_success ~ time_county + age + C(month) + C(account_status)', data=merged).fit()
print(big_logit.summary())

When `time_county`, `month`, and `account_status` are included along with `age`, `age` and `time_county` are still significant predictors. We can also look at other feature selection methods.

Logistic regression can also be used to [select features](https://towardsdatascience.com/feature-selection-with-pandas-e3690ad8504b) based on their importance in predicting at least one successful survey.

In [ ]:
reg = LogisticRegressionCV()
cat_cols = ['month', 'account_status', 'county']
num_cols = ['age', 'time_county']
d = defaultdict(LabelEncoder)
le_fit = merged[cat_cols].apply(lambda x: d[x.name].fit_transform(x))
X = pd.concat((merged[num_cols], le_fit), axis=1)
y = merged['one_success']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

reg.fit(X_train, y_train)
print("Best accuracy score using built-in LogisticRegCV: %f" % reg.score(X_test, y_test))
coef = pd.Series(reg.coef_.flatten(), index=X.columns)
imp_coef = coef.sort_values()

imp_coef.plot(kind="barh")
plt.title("Feature importance using Logistic Regression Model")
plt.tight_layout()
plt.show()

It appears that the account_status has the largest effect on whether a survey is successful.
`age` and `month` are discarded. Recall that `age` and `account_status` are correlated, however.

In [ ]:
merged.groupby('account_status')['age'].mean()

The 'Not Active' group is slightly older. A test of the equality of group means can confirm this. The null hypothesis is that the means are equal.

In [ ]:
active_age = merged.age[merged.account_status == 'Active']
notactive_age = merged.age[merged.account_status == 'Not Active']
print(CompareMeans.from_data(active_age, notactive_age).ttest_ind())

The p-value is close to zero, rejecting the null hypothesis that the means of the two groups are equal.
So there is a statistically significant difference between the ages of 'Active' and 'Not Active' recipients.

Let's now use a tree-based classifier for comparison.

In [ ]:
reg_extra_tree = ExtraTreesClassifier(n_estimators=10)
reg_extra_tree.fit(X_train, y_train)
feat_imp = pd.Series(
    reg_extra_tree.feature_importances_,
    index=X.columns
).sort_values()
print(f"Mean accuracy on test data is {reg_extra_tree.score(X_test, y_test)}")
feat_imp.plot(kind="barh")
plt.title("Feature importance using Extra Tree Classifier")
plt.tight_layout()
plt.show()

This time the model has `time_county` as the most important effect. All of the features are chosen by the
model, however. Note that there is randomness in the model, so sometimes the features
will be ranked differently across different runs. But the model will still choose features important for prediction.